In [38]:
import numpy as np
import pandas as pd
from sklearn.linear_model          import LogisticRegression
from sklearn.svm                   import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection       import train_test_split, cross_val_score, LeaveOneOut
from sklearn.datasets              import make_classification
from sklearn.preprocessing         import StandardScaler
from sklearn.metrics               import accuracy_score
from sklearn.base                  import clone
print('Packages Loaded, proceed')

Packages Loaded, proceed


I am having this issue where my testing accuracy is consistently higher than my training accuracy, even when my testing accuracy should not be high at all.  This jupyter notebook documents my process.  Not only should the testing accuracy be lower than the validation accuracy, but the testing accuracy should be significantly lower since I would expect the data with many features (2000 x 20) to fit a model that captures much of the noise amongst the redundant variables, even after the feature selection.



I am using sklearn in my other models in my projects, so it is likely I have a mistake that I have made elsewhere as well.  My first thought was that perhaps my model was accidentally overwriting the estimator/model and it was causing an artificially inflated accuracy because I was getting a different model each time for the test set (or likewise a deflated accuracy for the validation on the training set)

I have this problem relatively frequently and with several datasets, and it there are not a) separate distributions between training/testing set and b) a sample size issue.  I have tried varying the train/test split ratio, doing cross validation manually (in case there was a bug with cross_val_score), implementing different Machine Learning Algorithms (note the imports), and optimizing various other hyperparameters (especially with LogisticRegression() having so many).  Even when I change the hyperparameters, it doesn't seem to matter too much, and just changing the regularization term doesn't exactly translate to methods like LinearDiscriminantAnalysis (I use eigen solver) and RandomForestClassifier.

Any input or help on how to circumvent this issue I am having would be appreciated.

Create and Standardize Toy Data

In [11]:
x, y = make_classification(n_samples = 200, n_features = 20, n_informative = 10, flip_y = 0.1, class_sep = 0.5,random_state = 1)
x0 = scaler.fit_transform(x)
x0,y

(array([[-0.93813556, -0.85595284,  0.77637696, ..., -0.80338187,
         -0.81416873,  0.05104663],
        [-2.15195234, -0.85801456,  1.29040257, ..., -1.01328675,
          0.29514178, -0.38691768],
        [-0.98432038, -1.63938321,  0.3354797 , ..., -0.03998931,
          0.63793917, -0.93734998],
        ...,
        [-0.65248744,  1.20235574, -0.51231733, ..., -0.11017948,
         -0.17404411,  1.94768843],
        [-2.66028583, -0.96384227, -0.77210554, ..., -1.12739269,
         -1.50948637,  0.66149758],
        [-1.05319369, -1.48888213, -2.24927062, ...,  1.23250692,
         -2.51510765,  0.08806142]]),
 array([1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1,
        1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
   

Perform Hyperparameter Optimization

In [43]:
XTrain, XTest, YTrain, YTest = train_test_split(x0,y,test_size = 0.3, stratify = y, random_state = 8675309)

C = [1e-4, 1e-3, 0.01, 0.1, 1, 10, 100, 1000]
solver = ['saga','sag']

param_grid = {'C':C,
              'solver': solver,
             }

clf = GridSearchCV(LogisticRegression(random_state = 42), param_grid, scoring = 'accuracy', cv = LeaveOneOut())
clf.fit(XTrain,YTrain)
clf.best_params_

{'C': 0.1, 'solver': 'saga'}

Create the estimator with the parameters from hyperparameter optimization

In [44]:
log = LogisticRegression(C = 0.1, solver = 'saga') #as per best params

Compare the Cross-Validation Accuracy and the Testing Accuracy

In [45]:
clf = clone(log)
model_ = clf.fit(XTrain, YTrain)
y_pred = model_.predict(XTest)
print('LOOCV Accuracy on Training Data ONLY is: ', np.mean(cross_val_score(log,XTrain, YTrain, scoring = 'accuracy', cv = LeaveOneOut())))
print('Testing Accuracy is: ', accuracy_score(YTest, y_pred))

LOOCV Accuracy on Training Data ONLY is:  0.5928571428571429
Testing Accuracy is:  0.55


Looks normal, right? But if we compare the average accuracy over 1000 train/test splits...

In [74]:
acc = np.zeros((1000,))
for i in range(1000):
    XTrain, XTest, YTrain, YTest = train_test_split(x0,y,test_size = 0.3, stratify = y, random_state = i**2)
    alg = clone(log) #This is to make sure I am not accidentally fitting/predicting with a previously fitted estimator
    acc[i] = np.mean(cross_val_score(alg,XTrain, YTrain,scoring = 'accuracy', cv = LeaveOneOut()))
    
t_acc = np.zeros((1000,))
for i in range(1000):
    Model = [] #Reset each time
    XTrain, XTest, YTrain, YTest = train_test_split(x0,y,test_size = 0.3, stratify = y, random_state = i**2)
    alg = clone(log)
    Model = alg.fit(XTrain, YTrain)
    y_pred = Model.predict(XTest)
    t_acc[i] = accuracy_score(YTest, y_pred)
print('Cross Validation on Training Set Accuracy over 1000 iterations: ', np.mean(acc))
print('Testing Accuracy over 1000 iterations (same splits): ', np.mean(t_acc))

Cross Validation on Training Set Accuracy over 1000 iterations:  0.5822571428571428
Testing Accuracy over 1000 iterations (same splits):  0.5929666666666666


This is a little strange, the t_acc should not be larger than acc... is there any strange behavior looking at the individual data?

In [51]:
acc[0:10], t_acc[0:10]

(array([0.57857143, 0.51428571, 0.58571429, 0.61428571, 0.63571429,
        0.6       , 0.55714286, 0.57142857, 0.57857143, 0.58571429]),
 array([0.56666667, 0.65      , 0.6       , 0.63333333, 0.56666667,
        0.56666667, 0.65      , 0.66666667, 0.7       , 0.51666667]))

Not really.  What happens if I fit the estimator on the original train test split used to optimize the hyperparams, and then test a bunch of random test sets on this model?  Eventually there should be some test sets that perform much poorer 

In [81]:
clf = clone(log)
XTrain, XTest, YTrain, YTest = train_test_split(x0,y,test_size = 0.3, stratify = y, random_state = 8675309)
print('Cross Validation Accuracy on Original Split used in Hyperparam. Optimization: ',np.mean(cross_val_score(clf,XTrain, YTrain, scoring = 'accuracy', cv = LeaveOneOut()))) #verify previous result


Model_ = clf.fit(XTrain, YTrain) #This remains fixed, outside the loop

t_acc2 = np.zeros((1000,))
for i in range(1000):
    XTrain, XTest, YTrain, YTest = train_test_split(x0,y,test_size = 0.3, stratify = y, random_state = i**2)
    y_pred = Model_.predict(XTest)
    t_acc2[i] = accuracy_score(YTest, y_pred)

print('Test Accuracy with 1000 Test Sets all on the same model: ', np.mean(t_acc2))

Cross Validation Accuracy on Original Split used in Hyperparam. Optimization:  0.5928571428571429
Test Accuracy with 1000 Test Sets all on the same model:  0.6561500000000001


Alright.  This behavior could possibly be attributed to the fact that the accuracy across the board is low, because its not like the accuracy should be lower than 0.5.  What happens when we import synthetic data that can be much more easily classified?

Below is a synthetic dataset (can't be super explicit about the generator, but it shouldn't matter too much for this issue) with 2000 features and 200 samples.  The data was scaled using StandardScaler()

In [60]:
X = pd.read_csv('Total_Dataset.csv')
X = X.drop(columns = 'Unnamed: 0')
X.head()

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,1.727939,-0.408282,-0.702795,-0.709684,-0.548450,0.428379,1.498732,-0.775337,3.545692,-0.727709,...,-1.991115,-0.332513,0.080433,-0.964743,1.748908,-0.827986,0.301926,-0.688749,-0.668324,0.506503
1,-0.948129,-1.534414,-0.317488,-0.289633,-0.626682,1.084706,-0.821099,-0.744635,2.250363,-0.203531,...,-1.097877,0.322854,-0.238746,-0.220579,0.723101,-0.654636,1.379500,-0.322806,-1.212171,-0.301734
2,0.570152,0.527955,-1.158218,-0.198142,0.683628,0.860689,0.635202,0.239283,1.327666,-0.108473,...,-0.584675,-0.466286,2.254270,-0.163865,1.918743,0.823504,0.669029,-1.035258,0.283971,2.313788
3,0.889491,-0.467942,-0.665719,-0.870582,-0.212309,2.909781,0.980574,-1.173218,3.023919,-0.979268,...,0.185068,-0.264236,-0.235187,-0.305832,0.405041,-0.340313,2.758644,-0.896897,-0.917175,-0.152348
4,0.781059,1.080958,-1.088813,-0.584897,1.375659,-0.338295,0.665265,0.539406,-0.517239,-0.438098,...,0.281022,-0.573114,1.225223,-1.028783,-0.215001,1.411628,-0.073375,-1.208114,-0.205165,1.130371


The Class Membership is as follows:

In [66]:
Y = np.zeros((200,))
Y[0:100] = 1
Y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

After some feature selection (again, don't want to be super explicit but it shouldn't matter), I get 13 features left (200 x 13 dataset).

A disclaimer: The below data is a shuffled version of the above data

In [67]:
X0 = pd.read_csv('simData.csv')
X0 = X0.drop(columns = 'Unnamed: 0')
X0.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.124679,-0.511666,0.727900,0.240307,0.373149,-0.567941,-0.830887,-0.003841,0.070714,-0.815380,0.115287,-0.662071,0.670274
1,-0.925660,-0.243895,-1.397182,0.741588,0.515703,-0.904077,0.768483,-0.799157,0.808489,0.901693,-0.945580,-0.203934,-1.323254
2,-0.277331,-0.347020,0.773039,0.635408,0.357638,0.055553,-0.706853,-0.115096,0.609852,-0.402179,-0.203096,-0.627237,0.554816
3,1.355087,1.832806,-0.502055,0.519937,0.496318,1.309723,1.648091,1.416451,0.281562,1.501264,1.372674,1.911366,-0.475303
4,-0.656498,-0.147177,-0.347383,-0.792749,-1.088772,1.005990,-0.648401,-0.714435,-0.720153,-0.625142,-0.779137,0.053101,-0.425556


In [68]:
Y0 = pd.read_csv('simY.csv')
Y0 = Y0.drop(columns = 'Unnamed: 0')
Y0.head()

,0
0,1.0
1,1.0
2,1.0
3,0.0
4,0.0


I won't be demonstrating on the 200x2000 dataset for the sake of verbosity, since I primarily care about the data post-feature selection anyways.  If you would like to try it, it should be fairly plug-and-play friendly.

In [70]:
XTrain, XTest, YTrain, YTest = train_test_split(X0,Y0,test_size = 0.3, stratify = Y0, random_state = 42)

C = [1e-4, 1e-3, 0.01, 0.1, 1, 10, 100, 1000]
solver = ['saga','sag']

param_grid = {'C':C,
              'solver': solver,
             }

clf = GridSearchCV(LogisticRegression(random_state = 42), param_grid, scoring = 'accuracy', cv = LeaveOneOut())
clf.fit(XTrain,YTrain)
clf.best_params_

{'C': 0.01, 'solver': 'saga'}

In [72]:
lr = LogisticRegression(C = 0.01, solver = 'saga')

In [73]:
clf = clone(log)
model_ = clf.fit(XTrain, YTrain)
y_pred = model_.predict(XTest)
print('LOOCV Accuracy on Training Data ONLY is: ', np.mean(cross_val_score(lr,XTrain, YTrain, scoring = 'accuracy', cv = LeaveOneOut())))
print('Testing Accuracy is: ', accuracy_score(YTest, y_pred))

LOOCV Accuracy on Training Data ONLY is:  0.9571428571428572
Testing Accuracy is:  0.9833333333333333


This already doesn't bode well. What happens when we average it over 1000 iterations?

In [76]:
sim_acc = np.zeros((1000,))
for i in range(1000):
    XTrain, XTest, YTrain, YTest = train_test_split(X0,Y0,test_size = 0.3, stratify = Y0, random_state = i**2)
    alg = clone(log) #This is to make sure I am not accidentally fitting/predicting with a previously fitted estimator
    sim_acc[i] = np.mean(cross_val_score(alg,XTrain, YTrain,scoring = 'accuracy', cv = LeaveOneOut()))
    
sim_t_acc = np.zeros((1000,))
for i in range(1000):
    Model = [] #Reset each time
    XTrain, XTest, YTrain, YTest = train_test_split(X0,Y0,test_size = 0.3, stratify = Y0, random_state = i**2)
    alg = clone(log)
    Model = alg.fit(XTrain, YTrain)
    y_pred = Model.predict(XTest)
    sim_t_acc[i] = accuracy_score(YTest, y_pred)
print('Cross Validation on Training Set Accuracy over 1000 iterations: ', np.mean(sim_acc))
print('Testing Accuracy over 1000 iterations (same splits): ', np.mean(sim_t_acc))

Cross Validation on Training Set Accuracy over 1000 iterations:  0.9694071428571427
Testing Accuracy over 1000 iterations (same splits):  0.9705333333333331


In [79]:
clf2 = clone(log)
print('Cross Validation Accuracy on Original Split used in Hyperparam. Optimization: ',np.mean(cross_val_score(clf,XTrain, YTrain, scoring = 'accuracy', cv = LeaveOneOut()))) #verify previous result

XTrain, XTest, YTrain, YTest = train_test_split(X0,Y0,test_size = 0.3, stratify = Y0, random_state = 8675309)
Model_ = clf2.fit(XTrain, YTrain) #This remains fixed, outside the loop

sim_t_acc2 = np.zeros((1000,))
for i in range(1000):
    XTrain, XTest, YTrain, YTest = train_test_split(X0,Y0,test_size = 0.3, stratify = Y0, random_state = i**2)
    y_pred = Model_.predict(XTest)
    sim_t_acc2[i] = accuracy_score(YTest, y_pred)

print('Test Accuracy with 1000 Test Sets all on the same model: ', np.mean(sim_t_acc2))

Cross Validation Accuracy on Original Split used in Hyperparam. Optimization:  0.9785714285714285
Test Accuracy with 1000 Test Sets all on the same model:  0.9805999999999999
